In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import json

import re

import asyncio
from playwright.async_api import async_playwright
import tqdm.asyncio

# Subset to software

In [103]:
df = pd.read_csv("./websites.csv")

In [105]:
len(df)

66342

In [104]:
df.head(2)

,Unnamed: 0,CompanyName,Website,Specialties,Address,Industry,Unnamed: 6
0,1543,Add*Ed,http://www.addedrg.ca,['autism' 'intellectual disability' 'teacher t...,"{'city': 'Kingston', 'country': 'canada', 'cou...",Education Management,NaN
1,1952,"Addison AG Tech, LP",http://www.AddisonFund.com,[],"{'city': 'Addison', 'country': 'united states'...",Financial Services,NaN


In [4]:
df.tail(2)

,Unnamed: 0,CompanyName,Website,Specialties,Address,Industry,Unnamed: 6
66340,7404336,Wireframe Marketing,https://wireframemarketing.com,['SEO Services' 'Logo Design' 'Branding' 'Soci...,"{'city': 'REXBURG', 'country': 'united states'...",Marketing & Advertising,NaN
66341,7404394,Wirehound Networks,http://www.wirehoundnetworks.com,[],"{'city': 'Richfield', 'country': 'united state...",Computer & Network Security,NaN


In [8]:
df = df[df['Industry'] == 'Computer Software'].reset_index()
len(df)

6548

In [9]:
df.head(2)

,index,Unnamed: 0,CompanyName,Website,Specialties,Address,Industry,Unnamed: 6
0,11,5314,Sawa Impact Labs,https://sawa.gallery/,[],"{'city': 'Stanford', 'country': 'united states...",Computer Software,NaN
1,19,5505,OpenMI at the University of Michigan,https://maizepages.umich.edu/organization/openmi,[],"{'city': 'Ann Arbor', 'country': 'united state...",Computer Software,NaN


In [42]:
def parse_single_quote_json(json_str):
    json_str_fixed = json_str.replace("'", '"').replace("None", "null")
    try:
        return json.loads(json_str_fixed)
    except json.JSONDecodeError as e:
        print(f"Error parsing JSON: {e}")
        return None
    
def _get_country(_str):
    x = parse_single_quote_json(_str)
    if x is None:
        return None
    if 'country' not in x.keys():
        return None
    return x['country']
    

In [43]:
df['country'] = df.Address.apply(lambda x: _get_country(x))

Error parsing JSON: Expecting ',' delimiter: line 1 column 19 (char 18)
Error parsing JSON: Expecting ',' delimiter: line 1 column 19 (char 18)
Error parsing JSON: Invalid control character at: line 1 column 18 (char 17)
Error parsing JSON: Expecting ',' delimiter: line 1 column 18 (char 17)
Error parsing JSON: Expecting ',' delimiter: line 1 column 13 (char 12)
Error parsing JSON: Invalid control character at: line 1 column 12 (char 11)
Error parsing JSON: Invalid control character at: line 1 column 81 (char 80)


country is good (was checking if it's full of Chinese/Indian companies)

In [44]:
df.country.value_counts()

country
united states     4734
united kingdom     884
canada             392
australia          254
singapore          172
ireland             79
new zealand         26
Name: count, dtype: int64

In [47]:
df.Website.head(20)

0                                https://sawa.gallery/
1     https://maizepages.umich.edu/organization/openmi
2                                   https://siftmo.com
3                              https://www.aleios.com/
4                                    http://niftys.com
5                                 https://basewell.com
6                                    http://journey.io
7                             https://apple.co/3HcKeLx
8                               https://sqdc.avoyer.ca
9                             http://www.appcoderz.com
10                                  https://appdive.io
11                              https://greenpixie.com
12                                http://tryparade.com
13                       http://www.extendingfamily.us
14                                 https://nixmade.com
15                             https://www.jumproom.io
16                                    https://maple.so
17                         http://www.blueprintgtm.com
18        

In [102]:
df = df[df['country'] == 'united states']
len(df)

4734

# Scrape website

In [59]:
async def scrape_url(url):
    try:
        async with async_playwright() as p:
            browser = await p.chromium.launch()
            context = browser.new_context()
            page = await browser.new_page()
            
            await page.goto(url)
            
            await page.wait_for_load_state(timeout=20000)
            await page.keyboard.press('PageDown')
            await page.wait_for_load_state(timeout=20000)
            
            href = await page.evaluate('() => document.location.href')
            await page.wait_for_load_state(timeout=20000)
            await page.screenshot(path="screenshot.png", full_page=True)
            
            content = await page.content()
            await browser.close()
            return content
    except Exception as e:
        print(e)
        return None

In [60]:
url = 'https://maizepages.umich.edu/organization/openmi'

In [61]:
content = await scrape_url(url)

/home/krisztian/miniconda3/envs/all/lib/python3.12/inspect.py:479: RuntimeWarning: coroutine 'Browser.new_context' was never awaited
  def isframe(object):


In [84]:
async def limit_parallel(tasks, n=5):
    """
    Run up to n async tasks in parallel.

    :param n: Maximum number of concurrent tasks.
    :param tasks: List of coroutine functions to execute.
    :return: List of results from the tasks.
    """
    semaphore = asyncio.Semaphore(n)

    async def sem_task(task):
        async with semaphore:
            return await task

    # Wrap tasks with semaphore logic
    wrapped_tasks = [sem_task(task) for task in tasks]

    # Run all tasks concurrently
    return await tqdm.asyncio.tqdm.gather(*wrapped_tasks)

In [85]:
urls = df.Website.head(20)

In [95]:
tasks = [scrape_url(u) for u in urls]

In [96]:
results = await limit_parallel(tasks, 10)

  5%|███████████▌                                                                                                                                                                                                                           | 1/20 [00:02<00:45,  2.40s/it]

Page.goto: net::ERR_NAME_NOT_RESOLVED at https://nixmade.com/
Call log:
navigating to "https://nixmade.com/", waiting until "load"



 10%|███████████████████████                                                                                                                                                                                                                | 2/20 [00:02<00:21,  1.20s/it]

Page.goto: net::ERR_CERT_COMMON_NAME_INVALID at http://www.appcoderz.com/
Call log:
navigating to "http://www.appcoderz.com/", waiting until "load"



 45%|███████████████████████████████████████████████████████████████████████████████████████████████████████▉                                                                                                                               | 9/20 [00:07<00:09,  1.17it/s]

Page.goto: net::ERR_NAME_NOT_RESOLVED at http://www.extendingfamily.us/
Call log:
navigating to "http://www.extendingfamily.us/", waiting until "load"



100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:13<00:00,  1.44it/s]


In [97]:
results[0]

'<!DOCTYPE html><html class="yui3-js-enabled js-focus-visible js flexbox canvas canvastext webgl no-touch hashchange history draganddrop rgba hsla multiplebgs backgroundsize borderimage borderradius boxshadow textshadow opacity cssanimations csscolumns cssgradients cssreflections csstransforms no-csstransforms3d csstransitions video audio svg inlinesvg svgclippaths" data-js-focus-visible="" style=""><div id="yui3-css-stamp" style="position: absolute !important; visibility: hidden !important"></div><head>\n\n  <title>Squarespace - Website Expired</title>\n  <meta http-equiv="X-UA-Compatible" content="chrome=1">\n\n  <script src="//assets.squarespace.com/@sqs/polyfiller/1.6/legacy.js" nomodule="" type="text/javascript" crossorigin="anonymous"></script>\n  <script src="//assets.squarespace.com/@sqs/polyfiller/1.6/modern.js" type="text/javascript" crossorigin="anonymous"></script>\n  <script crossorigin="anonymous" src="//assets.squarespace.com/universal/scripts-compressed/extract-css-runt

In [98]:
def _estime_whole_df(secs):
    return secs / 20 * len(df) / 3600

In [100]:
_estime_whole_df(13)

1.1822777777777778

Going to
1. Set of async job for whole list
2. do md conversion
3. do prompt + evals for extracting jobs links / classifying pages / etc

page classifications:
- is it a valid htmls
- does it contains jobs?
    - Yes / list
    - Yes / open apply
    - No / with links
    - No / no links
- extract links

# Do markdown conversion

In [154]:
df2 = pd.read_csv("../data/01_subset_enriched.csv")

In [155]:
df.to_csv("../data/01_subset_enriched.csv", index=False)

In [156]:
df.head(2)

,CompanyName,Website,scrape_status,homepage_content
0,Sawa Impact Labs,https://sawa.gallery/,Success,"<!DOCTYPE html><html class=""yui3-js-enabled js..."
1,OpenMI at the University of Michigan,https://maizepages.umich.edu/organization/openmi,Success,"<!DOCTYPE html><html lang=""en""><head>\n <me..."


In [157]:
df.scrape_status.value_counts()

scrape_status
Success        3655
Failed         1071
Not Started       8
Name: count, dtype: int64

In [125]:
failed = df[df.scrape_status == 'Failed']

In [126]:
len(failed)

505

In [127]:
success = df[df.scrape_status== 'Success'].reset_index(drop=True)

In [128]:
len(success)

1722

In [129]:
success.head()

,CompanyName,Website,scrape_status,homepage_content
0,Sawa Impact Labs,https://sawa.gallery/,Success,"<!DOCTYPE html><html class=""yui3-js-enabled js..."
1,OpenMI at the University of Michigan,https://maizepages.umich.edu/organization/openmi,Success,"<!DOCTYPE html><html lang=""en""><head>\n <me..."
2,Nifty's,http://niftys.com,Success,"<!DOCTYPE html><html class=""no-js"" lang=""en-US..."
3,Basewell,https://basewell.com,Success,"<!DOCTYPE html><html lang=""en""><head><meta cha..."
4,Journey,http://journey.io,Success,"<!DOCTYPE html><html lang=""en""><head>\n <me..."


In [130]:
from jobsfinder.core import html2md

In [131]:
print(html2md(success.homepage_content.iloc[0]))



Squarespace \- Website Expired


 
Website ExpiredThis account has expired. If you are the site owner, click below to login.


In [132]:
df = success.copy()

In [135]:
len(df)

1722

In [136]:
from tqdm import tqdm

In [137]:
df['md'] = [html2md(x) for x in tqdm(df.homepage_content)]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1722/1722 [06:27<00:00,  4.44it/s]


In [138]:
df['html_length'] = df.homepage_content.apply(lambda x: None if x is None else len(x))
df['md_length'] = df.md.apply(lambda x: None if x is None else len(x))

In [139]:
df.html_length.describe()

count    1.722000e+03
mean     2.566247e+05
std      8.206222e+05
min      3.900000e+01
25%      2.496650e+04
50%      8.439500e+04
75%      2.130652e+05
max      1.529246e+07
Name: html_length, dtype: float64

In [140]:
df.md_length.describe()

count      1722.000000
mean       6392.870499
std       11975.955074
min           0.000000
25%        1345.250000
50%        4034.000000
75%        7348.000000
max      261102.000000
Name: md_length, dtype: float64

In [141]:
df.md_length.describe()

count      1722.000000
mean       6392.870499
std       11975.955074
min           0.000000
25%        1345.250000
50%        4034.000000
75%        7348.000000
max      261102.000000
Name: md_length, dtype: float64

In [142]:
df['md2html_ratio'] = df.html_length / df.md_length

In [143]:
df.md2html_ratio.describe()

/home/krisztian/projects/usergems-test/venv/lib/python3.12/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


count    1722.000000
mean             inf
std              NaN
min         1.385650
25%         8.929793
50%        20.265694
75%        54.344181
max              inf
Name: md2html_ratio, dtype: float64

In [144]:
sub1 = df[df.md_length > 20000]

In [145]:
len(sub1)

81

# Website Classifications

GPT pipelines

- Check if it's right website (i.e. not just cookie banner, or can't load page, etc.)
- Contains job classification
- Extract links
- Format job description

In [20]:
import pandas as pd
from jobsfinder.gpts import *
from jobsfinder.core import cost_so_far

In [4]:
df = pd.read_csv("../data/03_valid_website.csv")

In [5]:
df.head()

,CompanyName,Website,scrape_status,md_status,md,html_length,md_length,valid_website
0,Sawa Impact Labs,https://sawa.gallery/,Success,Success,Squarespace \- Website Expired\n\n\n \nWebsite...,6998,124,valid
1,OpenMI at the University of Michigan,https://maizepages.umich.edu/organization/openmi,Success,Success,OpenMI \- University of Michigan Maize Pages\n...,39237,784,valid
2,Nifty's,http://niftys.com,Success,Success,DNS points to prohibited IP \| niftys.com \| C...,5755,982,valid
3,Basewell,https://basewell.com,Success,Success,Basewell[About](/about)[Blog](/blog)[Sign in](...,322761,3504,valid
4,Journey,http://journey.io,Success,Success,"Journey – engage, close, and onboard customers...",218666,3466,valid


In [6]:
invalids = df[df.valid_website == 'invalid']

In [7]:
df.valid_website.value_counts()

valid_website
valid      1812
invalid     188
Name: count, dtype: int64

In [8]:
df = df[df.valid_website == 'valid']

In [9]:
df.head()

,CompanyName,Website,scrape_status,md_status,md,html_length,md_length,valid_website
0,Sawa Impact Labs,https://sawa.gallery/,Success,Success,Squarespace \- Website Expired\n\n\n \nWebsite...,6998,124,valid
1,OpenMI at the University of Michigan,https://maizepages.umich.edu/organization/openmi,Success,Success,OpenMI \- University of Michigan Maize Pages\n...,39237,784,valid
2,Nifty's,http://niftys.com,Success,Success,DNS points to prohibited IP \| niftys.com \| C...,5755,982,valid
3,Basewell,https://basewell.com,Success,Success,Basewell[About](/about)[Blog](/blog)[Sign in](...,322761,3504,valid
4,Journey,http://journey.io,Success,Success,"Journey – engage, close, and onboard customers...",218666,3466,valid


In [10]:
sample = df.sample(100)

In [11]:
results = await quickcases(jobs_status, sample.md.tolist())

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:18<00:00,  5.35it/s]


In [14]:
sample['result'] = results

In [15]:
sample.result.value_counts()

result
No jobs           78
Link to jobs      20
Job open apply     2
Name: count, dtype: int64

In [23]:
links = sample[sample.result == 'Link to jobs']

In [25]:
await jobs_status(links.md.iloc[3])

JobsClassification(reasoning='The website contains a link to a jobs page, but does not list any specific job openings on the current page.', classification='Link to jobs', link='https://jobs.lightsource.ai')

In [ ]:
sub = sample[sample.result == "Link to jobs"]

# Follow Links

In [82]:
from jobsfinder.gpts import follow_links
from tqdm import tqdm
import os

In [98]:
res = await follow_links("https://notion.so", "https://notion.so", [])

Starting next link https://notion.so
Scraping page
converting to md
judging website status
Status reasoning='The website contains a link to a careers page, but does not list any specific job openings on the current page.' classification='Link to jobs' link='https://www.notion.so/careers' titles=[]
Starting next link https://www.notion.so/careers
Scraping page
converting to md
judging website status
Status reasoning='The website explicitly lists multiple job openings across various departments, including titles and locations, which qualifies it as a Job list.' classification='Job list' link='' titles=['Business Systems, Finance', 'Business Systems, Marketing Technology', 'Customer Experience Specialist - Korean', 'Enterprise Technical Support, EMEA', 'Head of Support, EMEA', 'Launch Operations Program Manager', 'Technical Quality Program Manager', 'Customer Success Manager, EMEA', 'Customer Success Manager, Japan', 'Mid-Market Customer Success Manager', 'Scaled Customer Success Manager'

In [99]:
res

{'status': 'Job list',
 'titles': ['Business Systems, Finance',
  'Business Systems, Marketing Technology',
  'Customer Experience Specialist - Korean',
  'Enterprise Technical Support, EMEA',
  'Head of Support, EMEA',
  'Launch Operations Program Manager',
  'Technical Quality Program Manager',
  'Customer Success Manager, EMEA',
  'Customer Success Manager, Japan',
  'Mid-Market Customer Success Manager',
  'Scaled Customer Success Manager',
  'Scaled Customer Success Manager',
  'AI Product Engineer',
  'Business Intelligence Engineer, Marketing',
  'Data Engineer, Finance',
  'Data Engineer, Marketing & Sales',
  'Engineering Manager',
  'Engineering Manager, Web Presence',
  'QA Engineer, Automation',
  'Software Engineer, Android Platform',
  'Software Engineer, Backend',
  'Software Engineer, Calendar',
  'Software Engineer, Core Product',
  'Software Engineer, Core Product',
  'Software Engineer, Data Platform',
  'Software Engineer, Enterprise',
  'Software Engineer, Growth',

In [85]:
async def follow_from_url(url):
    return await follow_links(url, url, [])

In [80]:
sub.Website.tolist()

['http://www.unstructured.io/',
 'https://www.tripletsoft.com',
 'https://www.tigereye.com/',
 'http://www.lightsource.ai',
 'https://www.moonehq.com',
 'https://tooljet.com',
 'https://znest.com/',
 'http://www.elemeno.dev',
 'https://getmaestro.ai',
 'https://www.getkinn.com',
 'https://govalo.com',
 'https://productpair.co',
 'http://seymoregrp.com',
 'https://www.sonartalent.com/',
 'https://aviator.co',
 'https://www.firmstudio.io/',
 'https://ryzlabs.com/',
 'https://blockjoy.com',
 'http://www.dtmrecruiting.com',
 'https://virtualproducer.io']

In [88]:
await follow_from_url('https://www.tripletsoft.com')

Starting next link https://www.tripletsoft.com
Scraping page
converting to md
judging website status
Status reasoning="The website includes a 'Careers' section in the navigation, indicating a link to potential job opportunities, but does not list any specific jobs or mention current openings." classification='Link to jobs' link='/about-us/careers' titles=[]
Starting next link https://www.tripletsoft.com/about-us/careers
Scraping page
converting to md
judging website status
Status reasoning="The website explicitly lists multiple job openings with titles such as 'React.js Developer', 'QA Engineer', 'React Native Developer', and 'Java Spring Boot Developer'." classification='Job list' link='' titles=['React.js Developer', 'QA Engineer', 'React Native Developer', 'Java Spring Boot Developer']


{'status': 'Job list',
 'titles': ['React.js Developer',
  'QA Engineer',
  'React Native Developer',
  'Java Spring Boot Developer'],
 'history': ['/about-us/careers']}

In [92]:
df = pd.read_csv("../data/03_valid_website.csv")

In [93]:
len(df)

2000

In [94]:
df.head(2)

,CompanyName,Website,scrape_status,md_status,md,html_length,md_length,valid_website
0,Sawa Impact Labs,https://sawa.gallery/,Success,Success,Squarespace \- Website Expired\n\n\n \nWebsite...,6998,124,valid
1,OpenMI at the University of Michigan,https://maizepages.umich.edu/organization/openmi,Success,Success,OpenMI \- University of Michigan Maize Pages\n...,39237,784,valid


In [95]:
df.valid_website

0         valid
1         valid
2         valid
3         valid
4         valid
         ...   
1995      valid
1996      valid
1997      valid
1998      valid
1999    invalid
Name: valid_website, Length: 2000, dtype: object

In [51]:
tasks = [follow_from_url(url) for url in sub.Website]

In [52]:
results = await limit_parallel(tasks)

  0%|                                                                                                                                                                                                                                               | 0/20 [00:00<?, ?it/s]

Starting next link https://ryzlabs.com/
Starting next link https://aviator.co
Starting next link https://www.getkinn.com
Starting next link https://www.firmstudio.io/
Starting next link http://www.unstructured.io/
Status reasoning="The website provides a link to apply for roles, but does not list any specific job openings. Therefore, it falls under the category of 'Link to jobs.'" classification='Link to jobs' link='https://jobs.lever.co/RyzLabs'
Starting next link https://jobs.lever.co/RyzLabs
Status reasoning="The website includes a 'Careers' link in the Company section, indicating there may be job opportunities available, but does not list any specific jobs on the page itself." classification='Link to jobs' link='https://www.firmstudio.io/careers'
Starting next link https://www.firmstudio.io/careers
Status reasoning='The website contains a link to a careers page where jobs may be listed, but does not provide any specific job listings on the page itself.' classification='Link to jobs

  5%|███████████▌                                                                                                                                                                                                                           | 1/20 [00:09<03:05,  9.75s/it]

Status reasoning="The website explicitly lists multiple job openings, including titles such as 'Dev Leader', 'Ruby on Rails Developer', and 'Software Engineer'." classification='Job list' link=''
Starting next link http://seymoregrp.com
Status reasoning="The website has a 'Careers' link in the navigation, indicating a section for job opportunities, but does not list any specific jobs or mention current openings." classification='Link to jobs' link='./careers'
Starting next link ./careers


 15%|██████████████████████████████████▋                                                                                                                                                                                                    | 3/20 [00:11<00:46,  2.71s/it]

...

 55%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                                                                                                       | 11/20 [00:47<01:05,  7.33s/it]

Status reasoning='The website mentions current job openings and encourages users to provide their background for specific roles, indicating that there are actual jobs listed.' classification='Job list' link='https://www.dtmrecruiting.com/job-openings/'
Starting next link https://www.sonartalent.com/
Error code: 400 - {'error': {'message': "Invalid value for 'content': expected a string, got null.", 'type': 'invalid_request_error', 'param': 'messages.[1].content', 'code': None}}
Status reasoning='The website contains a link to a job postings page, which suggests that jobs are available, even if they are not listed directly on the page provided.' classification='Link to jobs' link='/job_postings/'
Starting next link /job_postings/
Page.goto: Protocol error (Page.navigate): Cannot navigate to invalid URL
Call log:
navigating to "/job_postings/", waiting until "load"

Error code: 400 - {'error': {'message': "Invalid value for 'content': expected a string, got null.", 'type': 'invalid_reque

 55%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                                                                                                       | 11/20 [01:36<01:18,  8.73s/it]


CancelledError: 

Error code: 400 - {'error': {'message': "Invalid value for 'content': expected a string, got null.", 'type': 'invalid_request_error', 'param': 'messages.[1].content', 'code': None}}
Error code: 400 - {'error': {'message': "Invalid value for 'content': expected a string, got null.", 'type': 'invalid_request_error', 'param': 'messages.[1].content', 'code': None}}
Error code: 400 - {'error': {'message': "Invalid value for 'content': expected a string, got null.", 'type': 'invalid_request_error', 'param': 'messages.[1].content', 'code': None}}
5 iterations did not succeed!
Starting next link https://znest.com/
Error code: 400 - {'error': {'message': "Invalid value for 'content': expected a string, got null.", 'type': 'invalid_request_error', 'param': 'messages.[1].content', 'code': None}}
Status reasoning='The website mentions helping jobseekers and employers and has a dedicated jobs section, but it does not list any specific job openings at this time.' classification='Link to jobs' link='

In [78]:
results = []

for url in tqdm(sub.Website):
    res = await follow_from_url(url)
    results.append(res)

  0%|                                                                                                                                                                                                                                               | 0/20 [00:00<?, ?it/s]

Starting next link http://www.unstructured.io/
Scraping page
converting to md
judging website status
Status reasoning="The website includes a 'Careers' link in the navigation, indicating there may be job opportunities available, but does not list any specific jobs or mention current openings." classification='Link to jobs' link='./careers'
Starting next link http://www.unstructured.io/careers
Scraping page
converting to md
judging website status


  5%|███████████▌                                                                                                                                                                                                                           | 1/20 [00:08<02:34,  8.11s/it]

Status reasoning='The website explicitly lists multiple job openings with titles and links to apply, which qualifies it as a Job list.' classification='Job list' link=''
Starting next link https://www.tripletsoft.com
Scraping page
converting to md
judging website status
Status reasoning="The website includes a 'Careers' section in the navigation, indicating a link to potential job opportunities, but does not list any specific jobs or mention current openings." classification='Link to jobs' link='/about-us/careers'
Starting next link https://www.tripletsoft.com/about-us/careers
Scraping page
converting to md
judging website status


 10%|███████████████████████                                                                                                                                                                                                                | 2/20 [00:14<02:03,  6.85s/it]

...

Status reasoning='The website states that there are no active job posts right now, which qualifies it as an empty list.' classification='No jobs' link=None
Starting next link https://govalo.com
Scraping page
converting to md
judging website status
Status reasoning="The website has a 'Careers' link in the footer, indicating potential job opportunities, but does not list any specific jobs or mention current openings." classification='Link to jobs' link='https://govalo.com/pages/careers'
Starting next link https://govalo.com/pages/careers
Scraping page
converting to md
judging website status


 55%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                                                                                                       | 11/20 [03:45<03:14, 21.56s/it]

Status reasoning="The website has a 'Careers' section but does not provide any specific job listings or mention current job openings, indicating that it is currently empty." classification='No jobs' link=''
Starting next link https://productpair.co
Scraping page
converting to md
judging website status
Status reasoning='The website includes a link to a careers page, but does not list any specific job openings or mention current roles.' classification='Link to jobs' link='https://productpair.com/careers'
Starting next link https://productpair.com/careers
Scraping page
Page.goto: Timeout 30000ms exceeded.
Call log:
navigating to "https://productpair.com/careers", waiting until "load"

converting to md
judging website status
Error code: 400 - {'error': {'message': "Invalid value for 'content': expected a string, got null.", 'type': 'invalid_request_error', 'param': 'messages.[1].content', 'code': None}}
Error code: 400 - {'error': {'message': "Invalid value for 'content': expected a string

 55%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                                                                                                       | 11/20 [05:35<04:34, 30.47s/it]


CancelledError: 

# Check jobs data

In [119]:
import pandas as pd

In [120]:
df = pd.read_csv("../data/04_jobs.csv")

In [121]:
df.head(2)

,CompanyName,Website,scrape_status,md_status,md,html_length,md_length,valid_website,history,status,error,jobs
0,Sawa Impact Labs,https://sawa.gallery/,Success,Success,Squarespace \- Website Expired\n\n\n \nWebsite...,6998,124,valid,"[""https://sawa.gallery/""]",No jobs,NaN,NaN
1,OpenMI at the University of Michigan,https://maizepages.umich.edu/organization/openmi,Success,Success,OpenMI \- University of Michigan Maize Pages\n...,39237,784,valid,"[""https://maizepages.umich.edu/organization/op...",No jobs,NaN,NaN


In [122]:
df.status.value_counts()

status
No jobs              1471
Loop detected         226
Job open apply         56
Job list               43
Max depth reached       6
Name: count, dtype: int64

In [123]:
sub = df[df.status=='Job list']

In [124]:
len(sub)

43

In [118]:
sub.jobs

180         ["Product Designer", "Applied AI Researcher"]
184     ["VP of Partnership and Growth", "Design Engin...
195                                         ["Developer"]
249     ["Senior Hardware Engineer", "Senior Software ...
696     ["Sr. Software Engineer (Full-Stack)", "Sr. So...
700     ["Sr. Software Engineer", "Jr. Software Engine...
732                                  ["Marketing Badass"]
755     ["Nail Tech", "Makeup Artist", "Hair Stylist",...
1246    ["AI Research Intern", "Machine Learning Engin...
Name: jobs, dtype: object

In [114]:
sub.history

180                 ["http://gantry.io", "/careers"]
184             ["http://edgeandnode.com", "/jobs/"]
195                ["https://hummi.app", "/careers"]
249                    ["https://www.paraform.com/"]
696              ["https://www.datapad.io", "/jobs"]
700     ["https://bloomsystems.io", "/careers.html"]
732        ["https://www.climatetechcocktails.com/"]
755                           ["https://bizinc.io/"]
1246            ["https://www.kaito.ai/", "/career"]
Name: history, dtype: object